In [1]:
import polars as pl
import datetime as dt

In [2]:
apple_watch_tag = "Cezary’s Apple Watch"

# experiment_start_date = "2025-05-12"
# experiment_end_date = "2025-06-06" # maybe add monday 7th

# additional_week_start_date = "2025-06-30"
# additional_week_end_date = "2025-07-06" # maybe add monday 7th

experiment_start_date = pl.datetime(2025, 5, 11, 16, time_zone="UTC")
experiment_end_date = pl.datetime(2025, 6, 8, 16, time_zone="UTC")
additional_week_start_date = pl.datetime(2025, 7, 6, 16, time_zone="UTC")
additional_week_end_date = pl.datetime(2025, 7, 13, 16, time_zone="UTC")

AWAKE_TAG = "HKCategoryValueSleepAnalysisAwake"
ASLEEP_TAG = "HKCategoryValueSleepAnalysisAsleepUnspecified"
ASLEEP_CORE_TAG = "HKCategoryValueSleepAnalysisAsleepCore"
ASLEEP_REM_TAG = "HKCategoryValueSleepAnalysisAsleepREM"
ASLEEP_DEEP_TAG = "HKCategoryValueSleepAnalysisAsleepDeep"
IN_BED_TAG = "HKCategoryValueSleepAnalysisInBed"

TREATMENT_WEEKS = [20, 22]
CONTROL_WEEKS = [21, 23, 27, 28, 29]

In [3]:
df = pl.read_csv("../data/raw/raw_export.csv")
df = df.filter(pl.col("source_name") == apple_watch_tag)
df = df.with_columns(
    pl.col("start_date").str.to_datetime().alias("start_date_parsed_utc"),
    pl.col("end_date").str.to_datetime().alias("end_date_parsed_utc"),
    pl.col("range").str.extract(r"(\d+) days (\d+):(\d+):(\d+)", 1).cast(pl.Int32).alias("days"),
    pl.col("range").str.extract(r"(\d+) days (\d+):(\d+):(\d+)", 2).cast(pl.Int32).alias("hours"),
    pl.col("range").str.extract(r"(\d+) days (\d+):(\d+):(\d+)", 3).cast(pl.Int32).alias("minutes"),
    pl.col("range").str.extract(r"(\d+) days (\d+):(\d+):(\d+)", 4).cast(pl.Int32).alias("seconds")
).with_columns([
    # Convert to total minutes
    (pl.col("days") * 24 * 60 +
     pl.col("hours") * 60 +
     pl.col("minutes") +
     pl.col("seconds") / 60.0).alias("duration_minutes")
])
df

type,source_name,source_version,unit,creation_date,start_date,end_date,value,timezone,range,start_date_parsed_utc,end_date_parsed_utc,days,hours,minutes,seconds,duration_minutes
str,str,str,str,str,str,str,str,str,str,"datetime[μs, UTC]","datetime[μs, UTC]",i32,i32,i32,i32,f64
"""HKCategoryTypeIdentifierSleepA…","""Cezary’s Apple Watch""","""8.3""",null,"""2022-02-22 10:54:08+02:00""","""2022-02-22 02:10:51+02:00""","""2022-02-22 02:31:21+02:00""","""HKCategoryValueSleepAnalysisAs…","""Europe/Berlin""","""0 days 00:20:30""",2022-02-22 00:10:51 UTC,2022-02-22 00:31:21 UTC,0,0,20,30,20.5
"""HKCategoryTypeIdentifierSleepA…","""Cezary’s Apple Watch""","""8.3""",null,"""2022-02-22 10:54:08+02:00""","""2022-02-22 03:13:51+02:00""","""2022-02-22 05:38:51+02:00""","""HKCategoryValueSleepAnalysisAs…","""Europe/Berlin""","""0 days 02:25:00""",2022-02-22 01:13:51 UTC,2022-02-22 03:38:51 UTC,0,2,25,0,145.0
"""HKCategoryTypeIdentifierSleepA…","""Cezary’s Apple Watch""","""8.3""",null,"""2022-02-22 10:54:08+02:00""","""2022-02-22 05:40:21+02:00""","""2022-02-22 10:54:51+02:00""","""HKCategoryValueSleepAnalysisAs…","""Europe/Berlin""","""0 days 05:14:30""",2022-02-22 03:40:21 UTC,2022-02-22 08:54:51 UTC,0,5,14,30,314.5
"""HKCategoryTypeIdentifierSleepA…","""Cezary’s Apple Watch""","""8.3""",null,"""2022-02-23 10:00:24+02:00""","""2022-02-23 04:41:21+02:00""","""2022-02-23 05:40:51+02:00""","""HKCategoryValueSleepAnalysisAs…","""Europe/Berlin""","""0 days 00:59:30""",2022-02-23 02:41:21 UTC,2022-02-23 03:40:51 UTC,0,0,59,30,59.5
"""HKCategoryTypeIdentifierSleepA…","""Cezary’s Apple Watch""","""8.3""",null,"""2022-02-23 10:00:24+02:00""","""2022-02-23 05:45:21+02:00""","""2022-02-23 06:28:51+02:00""","""HKCategoryValueSleepAnalysisAs…","""Europe/Berlin""","""0 days 00:43:30""",2022-02-23 03:45:21 UTC,2022-02-23 04:28:51 UTC,0,0,43,30,43.5
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""HKCategoryTypeIdentifierSleepA…","""Cezary’s Apple Watch""","""11.5""",null,"""2025-08-27 09:56:59+02:00""","""2025-08-27 09:22:22+02:00""","""2025-08-27 09:26:52+02:00""","""HKCategoryValueSleepAnalysisAs…","""Europe/Berlin""","""0 days 00:04:30""",2025-08-27 07:22:22 UTC,2025-08-27 07:26:52 UTC,0,0,4,30,4.5
"""HKCategoryTypeIdentifierSleepA…","""Cezary’s Apple Watch""","""11.5""",null,"""2025-08-27 09:56:59+02:00""","""2025-08-27 09:26:52+02:00""","""2025-08-27 09:28:22+02:00""","""HKCategoryValueSleepAnalysisAw…","""Europe/Berlin""","""0 days 00:01:30""",2025-08-27 07:26:52 UTC,2025-08-27 07:28:22 UTC,0,0,1,30,1.5
"""HKCategoryTypeIdentifierSleepA…","""Cezary’s Apple Watch""","""11.5""",null,"""2025-08-27 09:56:59+02:00""","""2025-08-27 09:28:22+02:00""","""2025-08-27 09:31:52+02:00""","""HKCategoryValueSleepAnalysisAs…","""Europe/Berlin""","""0 days 00:03:30""",2025-08-27 07:28:22 UTC,2025-08-27 07:31:52 UTC,0,0,3,30,3.5


In [4]:
main_experiment_df = df.filter(
    ((pl.col("start_date_parsed_utc") >= experiment_start_date) & (pl.col("end_date_parsed_utc") <= experiment_end_date))
    | ((pl.col("start_date_parsed_utc") >= additional_week_start_date) & (pl.col("end_date_parsed_utc") <= additional_week_end_date))
)
main_experiment_df = main_experiment_df.with_columns(
    pl.when(pl.col("start_date_parsed_utc").dt.hour() < 20)
    .then(pl.col("start_date_parsed_utc").dt.date())
    .otherwise(pl.col("start_date_parsed_utc").dt.date() + pl.duration(days=1))
    .alias("sleep_day")
).with_columns(
    pl.col("sleep_day").dt.week().alias("calendar_week")
).with_columns(
    pl.when(pl.col("calendar_week").is_in(TREATMENT_WEEKS))
    .then(pl.lit("treatment"))
    .when(pl.col("calendar_week").is_in(CONTROL_WEEKS))
    .then(pl.lit("control"))
    .otherwise(pl.lit("outside_experiment"))
    .alias("experiment_group")
)

In [5]:
additional_week_data = main_experiment_df.filter(
    (pl.col("sleep_day") >= pl.date(2025, 6, 3)) &
    (pl.col("sleep_day") <= pl.date(2025, 6, 7))
)

In [6]:
sleep_summary = main_experiment_df.group_by("sleep_day").agg([
    # Duration by each sleep tag (in minutes)
    pl.col("duration_minutes").filter(pl.col("value") == AWAKE_TAG).sum().alias("awake_duration_mins"),
    pl.col("duration_minutes").filter(pl.col("value") == ASLEEP_TAG).sum().alias("asleep_unspecified_duration_mins"),
    pl.col("duration_minutes").filter(pl.col("value") == ASLEEP_CORE_TAG).sum().alias("asleep_core_duration_mins"),
    pl.col("duration_minutes").filter(pl.col("value") == ASLEEP_REM_TAG).sum().alias("asleep_rem_duration_mins"),
    pl.col("duration_minutes").filter(pl.col("value") == ASLEEP_DEEP_TAG).sum().alias("asleep_deep_duration_mins"),

    # Total sleep duration (all asleep tags combined)
    pl.col("duration_minutes").filter(
        pl.col("value").is_in([ASLEEP_TAG, ASLEEP_CORE_TAG, ASLEEP_REM_TAG, ASLEEP_DEEP_TAG])
    ).sum().alias("total_sleep_duration_mins"),

    # Total time in bed (including awake time)
    pl.col("duration_minutes").sum().alias("total_time_in_bed_mins"),

    # Sleep period times
    pl.col("start_date_parsed_utc").min().alias("sleep_period_start"),
    pl.col("start_date_parsed_utc").max().alias("sleep_period_end"),
    pl.col("calendar_week").first(),
    pl.col("experiment_group").first(),
]).sort("sleep_day")
sleep_summary

sleep_day,awake_duration_mins,asleep_unspecified_duration_mins,asleep_core_duration_mins,asleep_rem_duration_mins,asleep_deep_duration_mins,total_sleep_duration_mins,total_time_in_bed_mins,sleep_period_start,sleep_period_end,calendar_week,experiment_group
date,f64,f64,f64,f64,f64,f64,f64,"datetime[μs, UTC]","datetime[μs, UTC]",i8,str
2025-05-12,46.0,0.0,240.5,84.0,45.0,369.5,415.5,2025-05-11 22:52:07 UTC,2025-05-12 05:44:37 UTC,20,"""treatment"""
2025-05-13,17.5,0.0,351.0,97.0,66.0,514.0,531.5,2025-05-12 20:48:43 UTC,2025-05-13 05:12:13 UTC,20,"""treatment"""
2025-05-14,11.0,0.0,265.5,93.5,48.0,407.0,418.0,2025-05-13 22:51:58 UTC,2025-05-14 05:17:28 UTC,20,"""treatment"""
2025-05-15,24.0,0.0,287.0,108.0,66.0,461.0,485.0,2025-05-14 21:37:25 UTC,2025-05-15 05:38:55 UTC,20,"""treatment"""
2025-05-16,54.0,84.0,301.5,76.0,76.5,538.0,592.0,2025-05-15 21:29:17 UTC,2025-05-16 07:21:02 UTC,20,"""treatment"""
…,…,…,…,…,…,…,…,…,…,…,…
2025-07-09,46.0,119.5,179.0,93.5,51.0,443.0,489.0,2025-07-09 00:31:42 UTC,2025-07-09 08:40:29 UTC,28,"""control"""
2025-07-10,8.5,0.0,269.5,93.5,53.0,416.0,424.5,2025-07-09 23:32:12 UTC,2025-07-10 06:32:12 UTC,28,"""control"""
2025-07-11,20.0,71.5,237.5,81.0,61.5,451.5,471.5,2025-07-10 23:27:20 UTC,2025-07-11 07:32:04 UTC,28,"""control"""


In [7]:
sleep_summary.write_csv("../data/processed/cleaned_sleep_data_experiment.csv")

## Filter dataset to check if the values are normally distributed


In [8]:
start = pl.datetime(2022, 9, 14, 16, time_zone="UTC")
end = pl.datetime(2025, 5, 10, 16, time_zone="UTC")

In [9]:
normal_df = (df.filter(
    (pl.col("start_date_parsed_utc") >= start) &
    (pl.col("end_date_parsed_utc") <= end)
).with_columns(
    pl.when(pl.col("start_date_parsed_utc").dt.hour() < 20)
    .then(pl.col("start_date_parsed_utc").dt.date())
    .otherwise(pl.col("start_date_parsed_utc").dt.date() + pl.duration(days=1))
    .alias("sleep_day")
).group_by("sleep_day").agg([
    pl.col("duration_minutes").filter(pl.col("value") == AWAKE_TAG).sum().alias("awake_duration_mins"),
    pl.col("duration_minutes").filter(pl.col("value") == ASLEEP_TAG).sum().alias("asleep_unspecified_duration_mins"),
    pl.col("duration_minutes").filter(pl.col("value") == ASLEEP_CORE_TAG).sum().alias("asleep_core_duration_mins"),
    pl.col("duration_minutes").filter(pl.col("value") == ASLEEP_REM_TAG).sum().alias("asleep_rem_duration_mins"),
    pl.col("duration_minutes").filter(pl.col("value") == ASLEEP_DEEP_TAG).sum().alias("asleep_deep_duration_mins"),
    # pl.col("duration_minutes").filter(pl.col("value") == IN_BED_TAG).sum().alias("in_bed_duration_mins"),

    # Total sleep duration (all asleep tags combined)
    pl.col("duration_minutes").filter(
        pl.col("value").is_in([ASLEEP_TAG, ASLEEP_CORE_TAG, ASLEEP_REM_TAG, ASLEEP_DEEP_TAG])
    ).sum().alias("total_sleep_duration_mins"),

    # Total time in bed (including awake time)
    pl.col("duration_minutes").sum().alias("total_time_in_bed_mins"),

    # Sleep period times
    pl.col("start_date_parsed_utc").min().alias("sleep_period_start"),
    pl.col("start_date_parsed_utc").max().alias("sleep_period_end"),
]).filter(
    # pl.col("in_bed_duration_mins") == 0  # filter out days with no "in bed" data since they occur when AW couldn't track properly
).sort("sleep_day"))

In [10]:
len(normal_df)

824

In [11]:
normal_df.write_csv("../data/processed/cleaned_normal_data_experiment.csv")